In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE273630"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE273630"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE273630.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE273630.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE273630.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Dopamine-regulated biomarkers in peripheral blood of HIV+ Methamphetamine users"
!Series_summary	"HIV and Methamphetamine study - Translational Methamphetamine AIDS Research Center - Dopamine-regulated inflammatory biomarkers"
!Series_summary	"A digital transcript panel was custom-made based on Hs_NeuroPath_v1 (Nanostring) to accommodate dopamine-regulated inflammatory genes that were previously identified in vitro, and hypothesized to cluster HIV+ Methamphetamine users."
!Series_overall_design	"Specimens were peripheral blood leukocytes isolated from participants that included adults enrolled by NIH-funded studies at the University of California San Diego’s HIV Neurobehavioral Research Program (HNRP) and Translational Methamphetamine Research Center (TMARC) under informed consent and approved protocols. The subset of PWH and PWoH selected for this study were by design males, between 35 – 44 years old, due to cohort characteristics and to increase

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the background information and sample characteristics
import pandas as pd
import os
import json

# 1. Gene Expression Data Availability
# Based on the series summary, this appears to be a custom digital transcript panel for
# dopamine-regulated inflammatory genes, which suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, there's not much data visible,
# but from the background information we can infer some details

# For trait (Epilepsy), the background mentions exclusion criteria including epilepsy,
# which suggests all participants do NOT have epilepsy
trait_row = None  # No information about epilepsy status for each individual

# For age, the background mentions participants were between 35-44 years old
# But this seems to be a selection criteria rather than variable data per participant
age_row = None  # No individual-level age data

# For gender, the background mentions participants were males
# But this seems to be constant across all participants
gender_row = None  # No individual-level gender data (all males)

# 2.2 Data Type Conversion Functions
# Even though we don't have these variables available, we'll define conversion functions
# in case they're needed for other processing

def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # For epilepsy trait
    value = value.lower()
    if 'epilepsy' in value or 'seizure' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        age = float(value)
        return age
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: female=0, male=1
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability (is_trait_available)
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None, which means clinical data is not available


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE273630/GSE273630_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE273630/GSE273630_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (780, 99)
First 20 gene/probe identifiers:
['ABAT', 'ABL1', 'ACAA1', 'ACHE', 'ACIN1', 'ACTN1', 'ACVRL1', 'ADAM10', 'ADCY5', 'ADCY8', 'ADCY9', 'ADCYAP1', 'ADORA1', 'ADORA2A', 'ADRA2A', 'ADRB2', 'AGER', 'AIF1', 'AKT1', 'AKT1S1']


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed in the gene expression data, 
# they appear to be standard human gene symbols (like ABAT, ABL1, etc.)
# which are the official symbols used in human genetics, not probe IDs or other identifiers.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (780, 99)
Gene data shape after normalization: (780, 99)
Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE273630.csv
No trait data (Epilepsy) available in this dataset based on previous analysis.
Cannot proceed with data linking due to missing trait or gene data.
Abnormality detected in the cohort: GSE273630. Preprocessing failed.
